第43回_Beginner限定コンペ】国勢調査からの収入予測
https://signate.jp/competitions/987/data

目的は収入予測の結果。国勢調査から、50,000ドル = 600万円くらいを超えるかどうかの２値を予想

やっていること
- sklaernでロジスティックでの二値分析

やること
- 基礎的な2値分析
- 基礎的なアルゴリズムを片っ端から試して値を予測 -> 判定
- y-dataで可視化したやつを研究

やったこと
- 環境構築
- y-dataの可視化

In [3]:
import pandas as pd
import numpy as np

/var/folders/rm/jy7mq9j90l58_8h20wjcxmxm0000gn/T/ipykernel_35981/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
from ydata_profiling import ProfileReport 

In [99]:
# CSVファイルのパス
train_path = "train.csv"
test_path = "test.csv"

# CSVファイルの読み込み
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
def get_unique_categories(df, column_name):
    # 指定されたカラムのユニークな値を取得してリストにする
    unique_categories = df[column_name].unique().tolist()

    # 結果を返す
    return unique_categories


# 'workclass'カラムのカテゴリ変数の種類を集計
workclass_categories = get_unique_categories(train_df, "workclass")
fnlwgt_categories = get_unique_categories(train_df, "fnlwgt")
education_categories = get_unique_categories(train_df, "education")
marital_status_categories = get_unique_categories(train_df, "marital-status")
occupation_categories = get_unique_categories(train_df, "occupation")
relationship_categories = get_unique_categories(train_df, "relationship")
race_categories = get_unique_categories(train_df, "race")
sex_categories = get_unique_categories(train_df, "sex")
native_country_categories = get_unique_categories(train_df, "native-country")


print(workclass_categories)
print(fnlwgt_categories)
print(education_categories)
print(marital_status_categories)
print(occupation_categories)
print(relationship_categories)
print(race_categories)
print(sex_categories)
print(native_country_categories)

In [101]:
train_df["workclass"].replace(
    [
        "Private",
        "State-gov",
        "?",
        "Local-gov",
        "Federal-gov",
        "Self-emp-inc",
        "Self-emp-not-inc",
    ],
    [0, 1, 2, 3, 4, 5, 6],
    inplace=True,
)

train_df["fnlwgt"].replace(
    [132652, 24694, 34572],
    [0, 1, 2],
)

train_df["education"].replace(
    [
        "Some-college",
        "HS-grad",
        "Bachelors",
        "Masters",
        "Prof-school",
        "12th",
        "Assoc-voc",
        "Assoc-acdm",
        "11th",
        "10th",
        "7th-8th",
        "5th-6th",
        "9th",
        "1st-4th",
    ],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    inplace=True,
)

train_df["marital-status"].replace(
    [
        "Married-civ-spouse",
        "Never-married",
        "Divorced",
        "Separated",
        "Widowed",
        "Married-spouse-absent",
        "Married-AF-spouse",
    ],
    [0, 1, 2, 3, 4, 5, 6],
    inplace=True,
)

train_df["occupation"].replace(
    [
        "Adm-clerical",
        "Prof-specialty",
        "Craft-repair",
        "Protective-serv",
        "Exec-managerial",
        "?",
        "Sales",
        "Handlers-cleaners",
        "Tech-support",
        "Machine-op-inspct",
        "Other-service",
        "Farming-fishing",
        "Transport-moving",
    ],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,],
    inplace=True,
)

train_df["relationship"].replace(
    [
        "Husband",
        "Not-in-family",
        "Own-child",
        "Unmarried",
        "Wife",
        "Other-relative",
    ],
    [0, 1, 2, 3, 4, 5],
    inplace=True,
)

train_df["race"].replace(
    [
        "White",
        "Black",
        "Asian-Pac-Islander",
        "Amer-Indian-Eskimo",
        "Other",
    ],
    [0, 1, 2, 3, 4],
    inplace=True,
)

train_df["sex"].replace(
    ["Female", "Male"],
    [0, 1],
    inplace=True,
)

train_df["native-country"].replace(
    ["United-States", "Mexico", "Philippines"],
    [0, 1, 2],
    inplace=True,
)

print(train_df)

       index  age  workclass  fnlwgt  education  education-num  \
0        322   21          0  132652          0             10   
1      11968   29          0  132652          1              9   
2      10868   19          0  132652          2             13   
3       3394   17          0  132652          1              9   
4      15993   47          0  132652          0             10   
...      ...  ...        ...     ...        ...            ...   
11895   9225   23          0  132652          1              9   
11896  13123   31          0  132652          1              9   
11897   9845   21          0  132652          1              9   
11898  10799   29          0  132652          2             13   
11899   2732   17          0  132652          1              9   

       marital-status  occupation  relationship  race  sex  native-country  Y  
0                   2           0             2     0    0               0  0  
1                   2           0             1

/var/folders/rm/jy7mq9j90l58_8h20wjcxmxm0000gn/T/ipykernel_35981/112548720.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df["workclass"].replace(
/var/folders/rm/jy7mq9j90l58_8h20wjcxmxm0000gn/T/ipykernel_35981/112548720.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df["workc

In [102]:
test_df["workclass"].replace(
    [
        "Private",
        "State-gov",
        "?",
        "Local-gov",
        "Federal-gov",
        "Self-emp-inc",
        "Self-emp-not-inc",
    ],
    [0, 1, 2, 3, 4, 5, 6],
    inplace=True,
)

test_df["education"].replace(
    [
        "Some-college",
        "HS-grad",
        "Bachelors",
        "Masters",
        "Prof-school",
        "12th",
        "Assoc-voc",
        "Assoc-acdm",
        "11th",
        "10th",
        "7th-8th",
        "5th-6th",
        "9th",
        "1st-4th",
    ],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    inplace=True,
)

test_df["marital-status"].replace(
    [
        "Married-civ-spouse",
        "Never-married",
        "Divorced",
        "Separated",
        "Widowed",
        "Married-spouse-absent",
        "Married-AF-spouse",
    ],
    [0, 1, 2, 3, 4, 5, 6],
    inplace=True,
)

test_df["occupation"].replace(
    [
        "Adm-clerical",
        "Prof-specialty",
        "Craft-repair",
        "Protective-serv",
        "Exec-managerial",
        "?",
        "Sales",
        "Handlers-cleaners",
        "Tech-support",
        "Machine-op-inspct",
        "Other-service",
        "Farming-fishing",
        "Transport-moving",
    ],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,],
    inplace=True,
)

test_df["relationship"].replace(
    [
        "Husband",
        "Not-in-family",
        "Own-child",
        "Unmarried",
        "Wife",
        "Other-relative",
    ],
    [0, 1, 2, 3, 4, 5],
    inplace=True,
)

test_df["race"].replace(
    [
        "White",
        "Black",
        "Asian-Pac-Islander",
        "Amer-Indian-Eskimo",
        "Other",
    ],
    [0, 1, 2, 3, 4],
    inplace=True,
)

test_df["sex"].replace(
    ["Female", "Male"],
    [0, 1],
    inplace=True,
)

test_df["native-country"].replace(
    ["United-States", "Mexico", "Philippines"],
    [0, 1, 2],
    inplace=True,
)

print(test_df)

      index  age  workclass  fnlwgt  education  education-num  marital-status  \
0      3873   17          3  132652          3             14               0   
1      3625   23          0  132652          0             10               0   
2      3028   19          0  132652          8              7               1   
3     13814   30          1  132652          1              9               1   
4     15398   60          0  132652          2             13               0   
...     ...  ...        ...     ...        ...            ...             ...   
5095   6726   19          0  132652          0             10               1   
5096   1501   23          0  132652          1              9               0   
5097   6391   36          0  132652         12              5               0   
5098   7104   33          0  132652          4             11               1   
5099   4214   36          3  132652          2             13               0   

      occupation  relations

/var/folders/rm/jy7mq9j90l58_8h20wjcxmxm0000gn/T/ipykernel_35981/706655261.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df["workclass"].replace(
/var/folders/rm/jy7mq9j90l58_8h20wjcxmxm0000gn/T/ipykernel_35981/706655261.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df["workcla

In [103]:
x_train = train_df.drop(['index','Y'], axis=1)  # 'Y'列を除外して特徴量を取得
y_train = train_df['Y']  

# 結果の確認
print(x_train.head())  # x_trainの最初の数行を表示
print(y_train.head(20))

   age  workclass  fnlwgt  education  education-num  marital-status  \
0   21          0  132652          0             10               2   
1   29          0  132652          1              9               2   
2   19          0  132652          2             13               1   
3   17          0  132652          1              9               1   
4   47          0  132652          0             10               0   

   occupation  relationship  race  sex  native-country  
0           0             2     0    0               0  
1           0             1     0    0               0  
2           1             1     0    0               0  
3           2             2     0    1               0  
4           2             0     0    1               0  
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     1
10    0
11    1
12    0
13    0
14    0
15    0
16    0
17    0
18    1
19    0
Name: Y, dtype: int64


In [104]:
# Pandas Profilingでレポートを作成
profile = ProfileReport(
    train_df,
    title="Pandas Profiling Report",
    explorative=True,
)

# レポートをHTMLファイルとして保存（または表示）
profile.to_file("report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [105]:
from sklearn.linear_model import LogisticRegression

In [112]:
clf = LogisticRegression(penalty="l2", solver="sag", random_state=0 ,class_weight='balanced')

In [113]:
clf.fit(x_train, y_train)

/Users/issei/miniforge3/envs/signate-43/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(class_weight='balanced', random_state=0, solver='sag')

In [114]:
x_test = test_df.drop(['index'], axis=1)  

In [115]:
y_pred = clf.predict(x_test)

In [117]:
print(y_pred[:20])

[0 0 0 1 1 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0]


In [118]:
output_df = test_df[['index']].copy()  # テストデータのindex列をコピー
output_df['Y'] = y_pred  # 予測結果を追加

# CSVファイルとして出力
output_file_path = 'predictions.csv'  # 出力ファイルのパス
output_df.to_csv(output_file_path, index=False)  # インデックスを含まずにCSVファイルに保存

print(f"予測結果が '{output_file_path}' に保存されました。")


予測結果が 'predictions.csv' に保存されました。
